# Miami Grand Prix
The Miami Grand Prix is a Formula One Grand Prix which was held for the first time during the 2022 season, with the event taking place at the Miami International Autodrome on a ten-year contract. 

In [ ]:
from urllib.request import urlopen
import json
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

## A little recap about what happened last year
Before to know what will happen in the grand prix, I consider that is neccesary to get to know what happened last year in order to have a reference to know what will happen this year in Miami.
In this case, I decided to take qualyfing and race as a reference to try to predict with the data obtained in the free practice how the weekend will unfold.

In [ ]:
response = urlopen("https://api.openf1.org/v1/sessions?year=2023&country_code=USA&circuit_short_name=Miami")
data = json.loads(response.read().decode('utf-8'))
calendar = pd.DataFrame(data)
calendar

In [ ]:
country_key='country_key=19'
country_code="country_code='USA'"
circuit_key='circuit_key=151'
year = 'year=2024'

### Qualyfing
In qualyfing,I will obtain the data related at the fastest driver on track to take as a reference his time and also his car data(RPM,speed,etc). With these data, it can see which gear will be used or at which speed a turn is taken.

In [ ]:
session_key='session_key=9074'
url = 'https://api.openf1.org/v1/laps?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
qualyfing = pd.DataFrame(data)
qualyfing.query("is_pit_out_lap== False")

In [ ]:
qualyfing.query("date_start > '2023-05-06T20:45:23.629000'").loc[qualyfing.query("date_start > '2023-05-06T20:45:23.629000'")['lap_duration'].idxmin()]

In [ ]:
from datetime import datetime, timedelta
startfastestlap = datetime(2023,5,6,20,50,13,99100)
startfastestlap+timedelta(seconds=86.841)

To obtain a clean analysis, seconds within his fastest lap will be taken to know how develop the car when it is in its maxium performance.

In [ ]:
driver_number='&driver_number=11'
url = "https://api.openf1.org/v1/car_data?"

urltotal = url+session_key+driver_number

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
fastestCarOnTrackData = pd.DataFrame(data)
fastestCarOnTrackData= fastestCarOnTrackData.query("date>='2023-05-06T20:50:13.991000' and date <= '2023-05-06T20:51:39.940100'").reset_index()

In [ ]:
plt.plot(fastestCarOnTrackData.index, fastestCarOnTrackData.n_gear,color="#001344")
plt.title("RPM")
plt.xlabel("x")
plt.ylabel("n_gear")
plt.show()

In [ ]:
plt.plot(fastestCarOnTrackData.index, fastestCarOnTrackData.speed,color="#001344")
plt.title("Speed")
plt.xlabel("x")
plt.ylabel("Speed (km/h)")
plt.show()

In [ ]:
plt.plot(fastestCarOnTrackData.index, fastestCarOnTrackData.throttle,color="#001344")
plt.title("Throttle")
plt.xlabel("x")
plt.ylabel("Throttle (km/h)")
plt.show()


## Race
Some things that it could be interesting to take into account it could be the time spent in the box stops and the race pace in each sector to compare with the data obtained in the race.

In [ ]:
session_key='session_key=9078'
url = 'https://api.openf1.org/v1/laps?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
race = pd.DataFrame(data)
race

In [ ]:
# Name of the drivers
url = 'https://api.openf1.org/v1/drivers?'
# In this case, due to Open URL does not refresh well the feature drivers, I will catch Practice 1 drivers as a reference.
#session_key = 'session_key=9472'
urltotal = url+session_key

response = urlopen(urltotal)
drivers = json.loads(response.read().decode('utf-8'))
drivers = pd.DataFrame(drivers)
# Restore the session key
#session_key='session_key=9481'
drivers

### Pits

In [ ]:
session_key='session_key=9078'
url = 'https://api.openf1.org/v1/pit?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
pits = pd.DataFrame(data)
pits

### Stints

In [ ]:
url = 'https://api.openf1.org/v1/stints?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
stintInformation = pd.DataFrame(data)
stintInformation

In [ ]:
def stint_configuration(stintInformation):
    stintsDataFrame = pd.DataFrame()
    for index,row in stintInformation.iterrows():
        number_driver = row.driver_number
        acronym_driver = drivers.query('driver_number == @number_driver').name_acronym.to_string(index=False)
        full_name = drivers.query('driver_number == @number_driver').full_name.to_string(index=False)
        team_name = drivers.query('driver_number == @number_driver').team_name.to_string(index=False)
        if row.lap_start != row.lap_end:
            contador = row.lap_start
            while contador <=row.lap_end :
                new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':contador,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
                stintsDataFrame = pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)
                contador+=1
        else:
        
            new_row = {'driver_number':row.driver_number,'compound':row.compound,'lap_number':row.lap_start,'name_acronym':acronym_driver,'full_name':full_name,'team_name':team_name}
            stintsDataFrame =pd.concat([stintsDataFrame, pd.DataFrame([new_row])], ignore_index=True)
    return stintsDataFrame

In [ ]:
stintsDataFrame =stint_configuration(stintInformation)
jointables2 = pd.merge(race,stintsDataFrame,on=['lap_number','driver_number'])
jointables2

### Fastest lap per compound

In [ ]:
compoundsPace = jointables2.loc[jointables2.groupby(['compound'])['lap_duration'].idxmin()]
compoundsPace

### Sector 1

In [ ]:
race_pace = pd.DataFrame(jointables2.query("is_pit_out_lap == False  and lap_duration <= 95").groupby("team_name")['duration_sector_1'].mean().sort_values(ascending=True))
race_pace

### Sector 2

In [ ]:
race_pace = pd.DataFrame(jointables2.query("is_pit_out_lap == False  and lap_duration <= 95").groupby("team_name")['duration_sector_2'].mean().sort_values(ascending=True))
race_pace

### Sector 3

In [ ]:
race_pace = pd.DataFrame(jointables2.query("is_pit_out_lap == False  and lap_duration <= 95").groupby("team_name")['duration_sector_3'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables2.query("is_pit_out_lap == False  and lap_duration <= 95").groupby("compound")['lap_duration'].mean().sort_values(ascending=True))
race_pace

# FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX
First of all it is neccesary to obtain all the data of the 2024 sessions in order to obtain some data as session_key,country_key,country_code and circuit_key

In [ ]:
response = urlopen("https://api.openf1.org/v1/sessions?year=2024&country_code=USA&circuit_short_name=Miami")
data = json.loads(response.read().decode('utf-8'))
calendar = pd.DataFrame(data)
calendar

## Setup

In [ ]:
session_key='session_key=9497'
country_key='country_key=19'
country_code="country_code='USA'"
circuit_key='circuit_key=151'
year = 'year=2024'

## Free Practice 1

In [ ]:
url = 'https://api.openf1.org/v1/laps?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
practice1 = pd.DataFrame(data)
practice1

In [ ]:
#Stints
url = 'https://api.openf1.org/v1/stints?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
stintInformation = pd.DataFrame(data)
stintInformation

In [ ]:
# Name of the drivers
url = 'https://api.openf1.org/v1/drivers?'
# In this case, due to Open URL does not refresh well the feature drivers, I will catch Practice 1 drivers as a reference.
#session_key = 'session_key=9472'
urltotal = url+session_key

response = urlopen(urltotal)
drivers = json.loads(response.read().decode('utf-8'))
drivers = pd.DataFrame(drivers)
# Restore the session key
#session_key='session_key=9481'
drivers

In [ ]:
stintsDataFrame =stint_configuration(stintInformation)
jointables2 = pd.merge(practice1,stintsDataFrame,on=['lap_number','driver_number'])
jointables2

### Data tyres

In [ ]:
def define_colour(compound):
    if compound == "SOFT":
        colour = "red"
    elif compound == "MEDIUM":
        colour = "yellow"
    elif compound == "HARD":
        colour = "grey"
    elif compound == "INTERMEDIATE":
        colour = "green"
    else:
        colour = "blue"
    return colour

In [ ]:
def show_plot(arrayDataframes,colour):
    figure, axis = plt.subplots(len(arrayDataframes),figsize=(15,85))
    #plt.xlim(92, 96)
    i=0
    for arr in arrayDataframes:
        arr.reset_index(drop=True, inplace=True)
        axis[(i)].plot( arr.lap_duration,marker ="o",color=colour,label = str(arr.full_name[0]) )
        axis[i].set_xlabel("Lap time")
        axis[i].set_ylabel("Lap Time Seconds")
        axis[i].legend()
        i = i+1

In [ ]:
def obtain_data_tyres(dataset,compound,race):
    extra = ''
    if race == False:
        extra = ' and lap_duration <97'
    else:
        extra = 'and lap_duration < 105'
    setTyres = dataset.query('compound == @compound '+extra)
    drivers_number = []
    set_dict = {}
    for index,row in setTyres.iterrows():
        if row.driver_number not in drivers_number:
            set_dict[row.driver_number] = []
            drivers_number.append(row.driver_number)

        set_dict[row.driver_number].append(row)

    arrayDataframes = []
    for valor in set_dict.values():
        arrayDataframes.append(pd.DataFrame(valor))
    colour = define_colour(compound)
    show_plot(arrayDataframes,colour)

In [ ]:
obtain_data_tyres(jointables2,"MEDIUM",False)

In [ ]:
obtain_data_tyres(jointables2,"SOFT",False)

In [ ]:
obtain_data_tyres(jointables2,"HARD",False)

In [ ]:
# Source: https://www.geeksforgeeks.org/how-to-annotate-bars-in-barplot-with-matplotlib-in-python/
def obtainchart(xvariable,yvariable,dataset):
    plt.figure(figsize=(12, 9))
    plots = sns.barplot(x=xvariable, y=yvariable, data=dataset,color='red')
    for bar in plots.patches:
        plots.annotate(format(bar.get_height(), '.3f'), 
                       (bar.get_x() + bar.get_width() / 2, 
                        bar.get_height()), ha='center', va='center',
                       size=8, xytext=(0, 7),
                       textcoords='offset points')
    plt.show()

### Top speed

In [ ]:
top_speed = jointables2.loc[jointables2.groupby(['name_acronym'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("name_acronym","st_speed",top_speed)

In [ ]:
top_speed = jointables2.loc[jointables2.groupby(['name_acronym'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("name_acronym","i1_speed",top_speed)

In [ ]:
top_speed = jointables2.loc[jointables2.groupby(['name_acronym'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("name_acronym","i2_speed",top_speed)

### Fastest lap per compound

In [ ]:
compoundsPace = jointables2.loc[jointables2.groupby(['compound'])['lap_duration'].idxmin()]
compoundsPace[['full_name','compound','duration_sector_1','duration_sector_2','duration_sector_3','lap_duration']]

### Fastest lap

In [ ]:
def obtain_fastest_lap(driver,dataset,newdataset):
    fastest_lap = dataset.query("driver_number == @driver").lap_duration.min()
    team_name = dataset.query("driver_number == @driver").head(1).team_name.to_string(index=False)
    #team_colour = dataset.query("driver_number == @driver").head(1).team_colour.to_string(index=False)
    name_acronym = dataset.query('driver_number == @driver').head(1).name_acronym.to_string(index=False)
    new_row = {'driver_number':driver,'fastest_lap':fastest_lap,'name_acronym': name_acronym, 'team_name':team_name}
    newdataset =pd.concat([newdataset, pd.DataFrame([new_row])], ignore_index=True)
    return newdataset

In [ ]:
def obtain_deltas(dataset,array):
    fastest_lap = dataset.fastest_lap.min()
    for row in dataset.iterrows():
        lap = row[1][1]
        delta = lap-fastest_lap
        array.append(delta)
    return array

In [ ]:
practiceCleaned = jointables2.query("lap_duration >1")
drivers_list = list(practiceCleaned['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_fastest_lap(driver,practiceCleaned,newdataset)

In [ ]:
arr= obtain_deltas(newdataset,[])
newdataset.insert(3,'delta',arr)

In [ ]:
dt = newdataset.sort_values(ascending=True,by='delta')
obtainchart("name_acronym","delta",dt)

### Track dominance

In [ ]:
sectorPace = jointables2.loc[jointables2.groupby(['driver_number'])['duration_sector_1'].idxmin()].sort_values(by=['duration_sector_1'],ascending=True)
sectorPace[['duration_sector_1','full_name','compound','lap_duration','lap_number']]

In [ ]:
sectorPace = jointables2.loc[jointables2.groupby(['driver_number'])['duration_sector_2'].idxmin()].sort_values(by=['duration_sector_2'],ascending=True)
sectorPace[['duration_sector_2','full_name','compound','lap_duration','lap_number']]

In [ ]:
sectorPace = jointables2.loc[jointables2.groupby(['driver_number'])['duration_sector_3'].idxmin()].sort_values(by=['duration_sector_3'],ascending=True)
sectorPace[['duration_sector_3','full_name','compound','lap_duration','lap_number']]

In [ ]:
race_pace = pd.DataFrame(jointables2.query("is_pit_out_lap == False  and lap_duration <= 95").groupby("compound")['lap_duration'].mean().sort_values(ascending=True))
race_pace

### Long laps

In [ ]:
def getinfolonglaps(dataset,driver_number,team,lap_duration_min=90,lap_duration_max=95):
    dataset = dataset.query("is_pit_out_lap == False and driver_number == @driver_number and team_name == @team and lap_duration < @lap_duration_max and lap_duration > @lap_duration_min ")
    return dataset[['full_name','compound','date_start','lap_number','duration_sector_1','duration_sector_2','duration_sector_3','lap_duration']]

#### Red Bull Racing

In [ ]:
stintInformation.query('driver_number == 1 or driver_number == 11')

In [ ]:
getinfolonglaps(jointables2,1,'Red Bull Racing',90,95)

In [ ]:
getinfolonglaps(jointables2,11,'Red Bull Racing',90,95)

#### Ferrari

In [ ]:
stintInformation.query('driver_number == 16 or driver_number == 55')

In [ ]:
getinfolonglaps(jointables2,55,'Ferrari',90,98)

#### McLaren

In [ ]:
stintInformation.query('driver_number == 4 or driver_number == 81')

In [ ]:
getinfolonglaps(jointables2,4,'McLaren',90,95)

In [ ]:
getinfolonglaps(jointables2,81,'McLaren',90,95)

#### Mercedes

In [ ]:
stintInformation.query('driver_number == 44 or driver_number == 63')

In [ ]:
getinfolonglaps(jointables2,44,'Mercedes',90,95)

In [ ]:
getinfolonglaps(jointables2,63,'Mercedes',90,95)

#### Aston Martin

In [ ]:
stintInformation.query('driver_number == 14 or driver_number == 18')

In [ ]:
getinfolonglaps(jointables2,14,'Aston Martin',90,95)

In [ ]:
getinfolonglaps(jointables2,18,'Aston Martin',90,95)

#### RB

In [ ]:
stintInformation.query('driver_number == 3 or driver_number == 22')

In [ ]:
getinfolonglaps(jointables2,3,'RB',90,95)

In [ ]:
getinfolonglaps(jointables2,22,'RB',90,95)

#### Williams

In [ ]:
stintInformation.query('driver_number == 2 or driver_number == 23')

In [ ]:
getinfolonglaps(jointables2,2,'Williams',90,95)

In [ ]:
getinfolonglaps(jointables2,23,'Williams',90,95)

#### Kick Sauber

In [ ]:
stintInformation.query('driver_number == 24 or driver_number == 77')

In [ ]:
getinfolonglaps(jointables2,24,'Kick Sauber',90,95)

In [ ]:
getinfolonglaps(jointables2,77,'Kick Sauber',90,95)

#### Haas

In [ ]:
stintInformation.query('driver_number == 20 or driver_number == 27')

In [ ]:
getinfolonglaps(jointables2,20,'Haas F1 Team',90,95)

In [ ]:
getinfolonglaps(jointables2,27,'Haas F1 Team',90,95)

#### Alpine

In [ ]:
stintInformation.query('driver_number == 10 or driver_number == 31')

In [ ]:
getinfolonglaps(jointables2,10,'Alpine',90,95)

In [ ]:
getinfolonglaps(jointables2,31,'Alpine',90,95)

## Sprint Qualyfing
### Race control
This section has been added in order to know which laps has been deleted and knowing what happened on track during this session as well.

In [ ]:
url = 'https://api.openf1.org/v1/race_control?'
session_key = 'session_key=9502'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
race_control = pd.DataFrame(data)
race_control

In [ ]:
url = 'https://api.openf1.org/v1/laps?'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
qualyfing = pd.DataFrame(data)
qualyfing

In race control dataset, I can see a lot of laptimes deleted, principally for track limits. Those laps deleted were deleted from dataset in order to obtain only the valid laps for the analysis.

In [ ]:
ids_deleted = [36,87,69,103,176]
for idv in ids_deleted:
    qualyfing = qualyfing.drop(idv)

In [ ]:
bestlap = qualyfing.loc[qualyfing.groupby(['driver_number'])['lap_duration'].idxmin()].sort_values(by=['lap_duration'],ascending=True)
bestlap[0:1]

The fastest lap  is 87.597 seconds (1.35.606= so that to obtain the competitve laps the fastest lap will be multiplied by 1.07 (93.72879 seconds) due to, according to the rules all the drivers have to do unless one lap within this gap.

In [ ]:
competitiveLaps = qualyfing.query("is_pit_out_lap == False and lap_duration < 93.72879")
competitiveLaps

In [ ]:
def obtain_information_qualy(driver,dataset):
    newdataset = None
    fastest_lap = dataset.query("driver_number == @driver").lap_duration.min()
    fastest_lap_absolute = dataset.lap_duration.min()
    delta = fastest_lap - fastest_lap_absolute
    st_speed = dataset.query("driver_number == @driver").st_speed.min()
    i1_speed = dataset.query("driver_number == @driver").i1_speed.min()
    i2_speed = dataset.query("driver_number == @driver").i2_speed.min()
    new_row = {'driver_number':driver,'fastest_lap':fastest_lap,'delta': delta,'st_speed':st_speed,'i1_speed':i1_speed,'i2_speed':i2_speed}
    newdataset =pd.concat([newdataset, pd.DataFrame([new_row])], ignore_index=True)
    return newdataset

In [ ]:
drivers_list = list(competitiveLaps['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_information_qualy(driver,competitiveLaps)
jointables = pd.merge(newdataset,drivers,on=['driver_number'])
jointables.sort_values(by=['fastest_lap'],ascending=True)

#### Best lap per driver compared with the best lap of the session in dry conditions

In [ ]:
obtainchart("name_acronym","delta",jointables.sort_values(by=['fastest_lap'],ascending=True))

In [ ]:
def obtain_difference_regard_reference(row,reference):
    newdataset = None
    difference_sector_1 = row.duration_sector_1 - reference.duration_sector_1.iloc[0]
    difference_sector_2 = row.duration_sector_2 - reference.duration_sector_2.iloc[0]
    difference_sector_3 = row.duration_sector_3 - reference.duration_sector_3.iloc[0]
    lap_duration = row.lap_duration - reference.lap_duration.iloc[0]
    new_row = {'driver_number':row.driver_number,'lap_duration':lap_duration,'difference_sector_1':difference_sector_1 ,'difference_sector_2':difference_sector_2,'difference_sector_3':difference_sector_3,'name_acronym':row.name_acronym   }
    
    newdataset =pd.concat([newdataset, pd.DataFrame([new_row])], ignore_index=True)
    return newdataset

In [ ]:
# Function done to obtain more information about the qualyfing session
def obtainInfoAboutQualySession(dataset,fecha):
    sessiondataset =dataset.query(fecha).sort_values(by='lap_duration')
    isFastestLap = []
    for index,row in sessiondataset.iterrows():
        driver = row.driver_number
        fastest_lap = sessiondataset.query("driver_number == @driver").lap_duration.min()
        if row.lap_duration == fastest_lap:
            isFastestLap.append(True)
        else:
            isFastestLap.append(False)
    sessiondataset['isFastestLap'] = isFastestLap
    return sessiondataset

In [ ]:
mergequaly = pd.merge(competitiveLaps,drivers,on=['driver_number'])
mergequaly

In [ ]:
# In order to know when each session finished, race control dataset will be consulted.
maximumDateQ1 = "date_start <'2024-05-03T20:49:01+00:00'"
maximumDateQ2 = "date_start <'2024-05-03T21:06:00+00:00' and date_start >='2024-05-03T20:49:01+00:00'"
maximumDateQ3 = "date_start >'2024-05-03T21:06:00+00:00'"

### Sprint Qualyfing 1

In [ ]:
q1Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ1).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q1Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P15 = q1Data[14:15]
P15

In [ ]:
print(
"Driver:",P15.full_name.to_string(index=False),
"Sector 1: ",P15.duration_sector_1.to_string(index=False),
"Sector 2: ",P15.duration_sector_2.to_string(index=False),
"Sector 3: ",P15.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q1Data[15::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P15,newdataset2)

newdataset2

### Best sector per driver

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

### Sprint Qualyfing 2

In [ ]:
q2Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ2).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q2Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P10 = q2Data[9:10]
print(
"Driver:",P10.full_name.to_string(index=False),
"Sector 1: ",P10.duration_sector_1.to_string(index=False),
"Sector 2: ",P10.duration_sector_2.to_string(index=False),
"Sector 3: ",P10.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q2Data[10::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P10,newdataset2)

newdataset2

### Best sector per driver

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

### Sprint Qualyfing 3

In [ ]:
q3Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ3).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q3Data

In [ ]:
#Reference
P1 = q3Data[:1]
print(
"Driver:",P1.full_name.to_string(index=False),
"Sector 1: ",P1.duration_sector_1.to_string(index=False),
"Sector 2: ",P1.duration_sector_2.to_string(index=False),
"Sector 3: ",P1.duration_sector_3.to_string(index=False)
)

##### Comparaison with poleman
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q3Data[1::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P1,newdataset2)
newdataset2

### Best sector per driver

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

### Best sector in the session

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

## Sprint

In [ ]:
session_key = 'session_key=9506'
url = 'https://api.openf1.org/v1/laps?'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
sprint = pd.DataFrame(data)
sprint

In [ ]:
#Stints
url = 'https://api.openf1.org/v1/stints?'

urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
stintInformation = pd.DataFrame(data)
stintInformation

In [ ]:
stintsDataFrame =stint_configuration(stintInformation)
jointables = pd.merge(race,stintsDataFrame,on=['lap_number','driver_number'])
jointables

In [ ]:
obtain_data_tyres(jointables,"MEDIUM",False)

In [ ]:
obtain_data_tyres(jointables,"SOFT",False)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("name_acronym","st_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("name_acronym","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("name_acronym","i2_speed",top_speed)

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 95").dropna().groupby("team_name")['lap_duration'].mean().sort_values(ascending=True))
race_pace

In [ ]:
def obtain_race_pace(dataset,array):
    fastest_lap = dataset.lap_duration.min()
    for row in dataset.iterrows():
        lap = row[1][0]
        delta = lap-fastest_lap
        array.append(delta)
    return array
arr= obtain_race_pace(race_pace,[])
race_pace.insert(1,'delta',arr)

### Race pace per teams

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 95").dropna().groupby("team_name")['duration_sector_1'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 95").dropna().groupby("team_name")['duration_sector_2'].mean().sort_values(ascending=True))
race_pace

In [ ]:
race_pace = pd.DataFrame(jointables.query("is_pit_out_lap == False  and lap_duration <= 95").dropna().groupby("team_name")['duration_sector_3'].mean().sort_values(ascending=True))
race_pace

#### Red Bull Racing

In [ ]:
stintInformation.query('driver_number == 1 or driver_number == 11')

In [ ]:
getinfolonglaps(jointables,1,'Red Bull Racing')

In [ ]:
getinfolonglaps(jointables,11,'Red Bull Racing')

#### Ferrari

In [ ]:
stintInformation.query('driver_number == 16 or driver_number == 55')

In [ ]:
getinfolonglaps(jointables,16,'Ferrari')

In [ ]:
getinfolonglaps(jointables,55,'Ferrari')

#### Mercedes

In [ ]:
stintInformation.query('driver_number == 63 or driver_number == 44')

In [ ]:
getinfolonglaps(jointables,63,'Mercedes')

In [ ]:
getinfolonglaps(jointables,44,'Mercedes')

#### Aston Martin

In [ ]:
stintInformation.query('driver_number == 14 or driver_number == 18')

In [ ]:
getinfolonglaps(jointables,14,'Aston Martin')

In [ ]:
getinfolonglaps(jointables,18,'Aston Martin')

#### McLaren

In [ ]:
stintInformation.query('driver_number == 4 or driver_number == 81')

In [ ]:
getinfolonglaps(jointables,81,'McLaren')

In [ ]:
getinfolonglaps(jointables,4,'McLaren')

#### Racing Bulls

In [ ]:
stintInformation.query('driver_number == 3 or driver_number == 22')

In [ ]:
getinfolonglaps(jointables,3,'RB')

In [ ]:
getinfolonglaps(jointables,22,'RB')

#### Williams

In [ ]:
stintInformation.query('driver_number == 2 or driver_number == 23')

In [ ]:
getinfolonglaps(jointables,2,'Williams')

In [ ]:
getinfolonglaps(jointables,23,'Williams')

#### Kick Sauber

In [ ]:
stintInformation.query('driver_number == 24 or driver_number == 77')

In [ ]:
getinfolonglaps(jointables,24,'Kick Sauber')

In [ ]:
getinfolonglaps(jointables,77,'Kick Sauber')

#### Alpine

In [ ]:
getinfolonglaps(jointables,10,'Alpine')

In [ ]:
getinfolonglaps(jointables,31,'Alpine')

#### Haas

In [ ]:
stintInformation.query('driver_number == 20 or driver_number == 27')

In [ ]:
getinfolonglaps(jointables,20,'Haas F1 Team')

In [ ]:
getinfolonglaps(jointables,27,'Haas F1 Team')

## Qualyfing
### Set up
First of all, it is neccesary to obtain the data about the qualyfing
#### Race control
This section has been added in order to know which laps has been deleted and knowing what happened on track during this session as well.

In [ ]:
url = 'https://api.openf1.org/v1/race_control?'
session_key = 'session_key=9498'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
race_control = pd.DataFrame(data)
race_control

### Laps

In [ ]:
url = 'https://api.openf1.org/v1/laps?'
#session_key = 'session_key=9492'
urltotal = url+session_key

response = urlopen(urltotal)
data = json.loads(response.read().decode('utf-8'))
qualyfing = pd.DataFrame(data)
qualyfing

As the qualyfing session has been without weather changes, I will take the fastest lap as a reference to obtain all the competitive laps.

In [ ]:
bestlap = qualyfing.loc[qualyfing.groupby(['driver_number'])['lap_duration'].idxmin()].sort_values(by=['lap_duration'],ascending=True)
bestlap[0:1]

In this case, the fastest lap is 87.241 seconds (1.27.241= so that to obtain the competitve laps the fastest lap will be multiplied by 1.07 (93.347 seconds) due to, according to the rules all the drivers have to do unless one lap within this gap.

In [ ]:
competitiveLaps = qualyfing.query("is_pit_out_lap == False and lap_duration < 93.347")
competitiveLaps

In [ ]:
drivers_list = list(competitiveLaps['driver_number'].unique())
newdataset = pd.DataFrame()
for driver in drivers_list:
    newdataset =obtain_information_qualy(driver,competitiveLaps)
jointables = pd.merge(newdataset,drivers,on=['driver_number'])
jointables.sort_values(by=['fastest_lap'],ascending=True)

#### Best lap per driver compared with the best lap of the session

In [ ]:
obtainchart("name_acronym","delta",jointables.sort_values(by=['fastest_lap'],ascending=True))

#### Speed trap pace per driver

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("name_acronym","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("name_acronym","i2_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['name_acronym'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("name_acronym","st_speed",top_speed)

### Speed trap pace per team

In [ ]:
top_speed = jointables.loc[jointables.groupby(['team_name'])['i1_speed'].idxmax()].sort_values(by=['i1_speed'],ascending=False)
obtainchart("team_name","i1_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['team_name'])['i2_speed'].idxmax()].sort_values(by=['i2_speed'],ascending=False)
obtainchart("team_name","i2_speed",top_speed)

In [ ]:
top_speed = jointables.loc[jointables.groupby(['team_name'])['st_speed'].idxmax()].sort_values(by=['st_speed'],ascending=False)
obtainchart("team_name","st_speed",top_speed)

In [ ]:
mergequaly = pd.merge(competitiveLaps,drivers,on=['driver_number'])
mergequaly

In [ ]:
# In order to know when each session finished, race control dataset will be consulted.
maximumDateQ1 = "date_start <'2024-05-04T20:25:01+00:00'"
maximumDateQ2 = "date_start <'2024-05-04T20:48:00+00:00' and date_start >'2024-05-04T20:25:01+00:00'"
maximumDateQ3 = "date_start >'2024-05-04T20:48:00+00:00'"

### Qualyfing 1

In [ ]:
q1Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ1).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q1Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P15 = q1Data[14:15]
P15

In [ ]:
print(
"Driver:",P15.full_name.to_string(index=False),
"Sector 1: ",P15.duration_sector_1.to_string(index=False),
"Sector 2: ",P15.duration_sector_2.to_string(index=False),
"Sector 3: ",P15.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q1Data[15::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P15,newdataset2)

newdataset2

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q1Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

#### Qualyfing 2

In [ ]:
q2Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ2).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q2Data

##### Comparaison with driver at risk
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P10 = q2Data[9:10]
print(
"Driver:",P10.full_name.to_string(index=False),
"Sector 1: ",P10.duration_sector_1.to_string(index=False),
"Sector 2: ",P10.duration_sector_2.to_string(index=False),
"Sector 3: ",P10.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q2Data[10::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P10,newdataset2)

newdataset2

#### Best sector per driver

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q2Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

#### Qualyfing 3

In [ ]:
q3Data = obtainInfoAboutQualySession(mergequaly,maximumDateQ3).query("isFastestLap == True").sort_values(by=['lap_duration'],ascending=True)
q3Data

##### Comparaison with poleman
In this section with the fastest lap done for each driver (laptimes deleted will not be taken into account to do this analysis) it will be a comparaison in order to see where the driver eliminated lost/gain time in their fastest lap.

In [ ]:
#Reference
P1 = q3Data[:1]
print(
"Driver:",P1.full_name.to_string(index=False),
"Sector 1: ",P1.duration_sector_1.to_string(index=False),
"Sector 2: ",P1.duration_sector_2.to_string(index=False),
"Sector 3: ",P1.duration_sector_3.to_string(index=False)
)

In [ ]:
newdataset2 = pd.DataFrame()
for index,row in q3Data[1::].iterrows():
    newdataset2 = obtain_difference_regard_reference(row,P1,newdataset2)
newdataset2

#### Best sector per driver

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(q3Data.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))

### Best lap of the session

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_1'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_2'].min().sort_values(ascending=True))

In [ ]:
pd.DataFrame(mergequaly.groupby("name_acronym")['duration_sector_3'].min().sort_values(ascending=True))